# Swept-power spectrum monitoring
### Data file summary

In [ ]:
# This notebook supports input parameters for automatic report generation. The parameters must be variables in this
# cell, which has a special 'parameters' tag.
DATA_ROOT = r'G:\Shared drives\Covid-19 Spectrum Monitoring\Data'

HISTOGRAM_RESOLUTION_SWEEPS = 100
HISTOGRAM_POWER_LOW = -110
HISTOGRAM_POWER_HIGH = -20

# report.py will make separate reports for each combination of the below parameters
# that exists in the dataset.
site = 'dUsVcuPP'
year = 2020
month = 7
day = 1

figure_format = 'png'

In [ ]:
# hide the pink warnings in reports (comment to include them)
import warnings
warnings.simplefilter("ignore")
from environment import *
import figures

set_matplotlib_formats(figure_format)

display(widgets.HTML(f'This report was produced {time.strftime(time_format)}'))

### Source data

In [ ]:
# metadata = read_dat.yaml_metadata(path)

# add metadata to the environment as variables
# globals().update(metadata)

start_date = pd.Timestamp(year=int(year), month=int(month), day=int(day))

hists = pd.read_parquet(
    Path(DATA_ROOT)/'histogram.parquet',
    filters=[
        ('Site', '=', site),
        ('Time', '>=', start_date),
        ('Time', '<', start_date + pd.DateOffset(days=1))
    ],
    use_legacy_dataset=False,
    use_threads=False
)

# select site
hists.reset_index('Site', drop=True, inplace=True)
hists.columns = hists.columns.astype('float32')

center_frequencies = hists.index.levels[0].values

# hists = pd.read_hdf(
#     Path(path).with_name('histogram.hdf'),
#     key='.'.join(Path(path).name.split('.')[:-2])
# )

# # Set the indexing to (Frequency, Time)
# hists = hists.reset_index().set_index(['Frequency', 'Time'])
# hists.columns = hists.columns.astype('float64')

# load the summary into a dataframe so the notebook shows it prettily
overview = pd.DataFrame(
    [{
        'Average sweep time': f'{(hists.index[-1][1]-hists.index[0][1]).total_seconds()/hists.shape[0]:0.1f}s',
#         'Dwell window length': f'{dwell_time:0.2f}s',
        'Frequency points': len(center_frequencies),
        'Sweep count': hists.shape[0],
        'Start': hists.index.get_level_values('Time').min().strftime(time_format),
        'End':hists.index.get_level_values('Time').max().strftime(time_format)
    }],
    index=['']
).T

# # there is slight error in the achieved frequency; map intended nearby frequency
# # to achieved frequency
fc_map = dict(zip(np.round(hists.index.levels[0],1),hists.index.levels[0]))

display(overview)

### LTE Uplink Bands

In [ ]:
for fc in [701.5, 709, 782, 821.3, 842.5]:
    ax = figures.plot_power_histogram_heatmap(
        hists.loc[fc_map[fc]], fc, 
    )

### LTE Downlink Bands

In [ ]:
for fc in [734, 739, 751, 866.3, 887.5]:
    ax = figures.plot_power_histogram_heatmap(
        hists.loc[fc_map[fc]], fc, 
    )

### 2.4 GHz ISM Band

In [ ]:
for fc in [2412, 2437, 2462]:
    ax = figures.plot_power_histogram_heatmap(
        hists.loc[fc_map[fc]], fc, 
    )

### 5 GHz U-NII1 Bands

In [ ]:
for fc in [5170,5190,5210, 5230, 5240, 5775, 5795]:
    ax = figures.plot_power_histogram_heatmap(
        hists.loc[fc_map[fc]], fc, 
    )

## Validation Checks
### Quiet band readings
The the 2695 MHz band allocation is protected from transmission for sensitive radioastronomy measurements, so its behavior should be similar to that of the calibration noise floor. On our hardware, this is typically around -105 dBm/4 MHz.

A simple time series histogram gives a global view on the noise distribution. Thermal noise in the receiver is be single-moded, so a bimodal distribution suggests the presence of out-of-band signal overload.

In [ ]:
aperture_time = 0.5e-3
sample_rate = 4e6

noise_hist = hists.loc[2695].sum(axis=0)
noise_hist = noise_hist[noise_hist>0]

if noise_hist.size > 0:
    fig, ax = plt.subplots()

    ax.hist(
        x=noise_hist.index,
        bins=len(noise_hist),
        weights=noise_hist.values,
        range=(noise_hist.index[0], noise_hist.index[-1]),
    #     cumulative=-1,
        density=True,
    )
    ax.set_title('2695 MHz')
    ax.set_yscale('log')
    ax.set_xlabel(f'Average power in {aperture_time/1e-3:0.1f}ms (dBm/{sample_rate/1e6:0.0f} MHz)')
    ax.set_ylabel(rf'Fraction of samples < abscissa ($N={noise_hist.sum()}$)');

The distribution by time helps identify transients in the distribution.  A steady noise floor stronger than -105 dBm/4 MHz may indicate that the noise calibration needs to be repeated, or the presence of steady, powerful signal overload in another band. Intermittent samples above this level suggest intermittent overload in another band.

In [ ]:
ax = figures.plot_power_histogram_heatmap(
    hists.loc[fc_map[2695]], 2695, bounds=(-140,-60)
)

In [ ]:
# ### IQ sample peak
# We expect undesirable compression effects when power is sustained at -30 dBm/4 MHz or above. We are not yet sure what peak IQ sample level will correspond with compression in this dataset.

# # plot the maximum instantaneous sample power at each frequency
# fig, ax = subplots()
# sample_maxima = peak_power.reset_index().pivot(columns='Frequency', values='Sample peak').max(axis=0)
# (10*log10(sample_maxima)).plot(lw=0,marker='.',ax=ax)
# ylabel(f'Max sample power (dBm/{int(metadata["sample_rate"]/1e6)} MHz)')
# xlabel('Center frequency (MHz)');
# pagebreak()